# Try PP-Picodet

In [1]:
import sys
from pathlib import Path
sys.path.insert(0,"..")

import numpy as np
import torch
import mmdet
import mmcv
import pandas as pd

from mmcv import Config
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
from mmdet.models import build_detector
from mmdet.apis import init_detector, inference_detector, single_gpu_test
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from src.models import build_detector

In [3]:
cfg.data.test

{'type': 'CocoDataset',
 'ann_file': 'data/coco/annotations/instances_val2017.json',
 'img_prefix': 'data/coco/images/val2017/',
 'pipeline': [{'type': 'LoadImageFromFile', 'to_float32': True},
  {'type': 'MultiScaleFlipAug',
   'img_scale': (320, 320),
   'flip': False,
   'transforms': [{'type': 'Resize', 'keep_ratio': False},
    {'type': 'RandomFlip'},
    {'type': 'Normalize',
     'mean': [123.675, 116.28, 103.53],
     'std': [58.395, 57.12, 57.375],
     'to_rgb': True},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'ImageToTensor', 'keys': ['img']},
    {'type': 'Collect', 'keys': ['img']}]}]}

In [2]:
config_file = "../configs/picodet/picodet_s_320_coco.py"
checkpoint_file = "../models/picodet_s_320.26.9.pth"

cfg = Config.fromfile(config_file)

# build the dataloader
dataset = build_dataset(cfg.data.test)
# data_loader = build_dataloader(
#     dataset,
#     samples_per_gpu=samples_per_gpu,
#     workers_per_gpu=cfg.data.workers_per_gpu,
#     dist=distributed,
#     shuffle=False
# )

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_file, map_location='cpu')
model.CLASSES = checkpoint["meta"]["CLASSES"]
model = MMDataParallel(model, device_ids=[0])

outputs = single_gpu_test(model, data_loader, args.show, args.show_dir,
                          args.show_score_thr)

loading annotations into memory...


FileNotFoundError: CocoDataset: [Errno 2] No such file or directory: 'data/coco/annotations/instances_val2017.json'

In [34]:
inp = np.random.rand(1, 320, 320,3)

In [38]:
_img_metas = {
    'ori_shape': (320, 320, 3),
    'img_shape': (320, 320, 3),
    'pad_shape': (320, 320, 3),
    'scale_factor': [1., 1., 1., 1.],
    'flip': True,
    'flip_direction': 'horizontal',
    'img_norm_cfg': {
        #'mean': [123.675, 116.28 , 103.53 ],
        #'std': [58.395, 57.12 , 57.375],
        'mean':[0., 0., 0.],
        'std':[1., 1., 1.],
        'to_rgb': True
    },
    'batch_input_shape': (320, 320)
    }

img_metas = [_img_metas] * inp.shape[0]


In [45]:
m.zero_grad()

In [47]:
m(inp, img_metas)

TypeError: forward_train() missing 2 required positional arguments: 'gt_bboxes' and 'gt_labels'

In [32]:
dir(m)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
